In [9]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
from livecellx import core
from livecellx.core import datasets
from livecellx.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecellx.core import SingleCellTrajectory, SingleCellStatic
import numpy as np
import os, json, cv2, random
import cv2

## SingleCellStatic: saving & loading 

In [10]:
io_out_dir = Path("test_io_output")

In [11]:
dataset_dir_path = Path(
    "../datasets/test_data_STAV-A549/DIC_data"
)

mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")


In [12]:
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")

time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

3 png img file paths loaded;


In [13]:
from skimage.measure import regionprops
from livecellx.core.io_sc import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

100%|██████████| 3/3 [00:06<00:00,  2.25s/it]


In [14]:
single_cells[1].meta

{'label_in_mask': 2}

### Writing single cells to json  
You can check the documentation of `write_single_cells_json` for more details.  
`return_list` controls whether or not to return json object. 

In [15]:
SingleCellStatic.write_single_cells_json(single_cells, path=io_out_dir/"single_cells_0.json")

In [16]:
sc_json_list = SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells_1.json", dataset_dir=io_out_dir/"dataset", return_list=True)

In [17]:
SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells_2.json", dataset_dir=io_out_dir/"dataset", return_list=False)

In [18]:
img_dataset = single_cells[0].img_dataset

In [19]:
loaded_scs = SingleCellStatic.load_single_cells_json(io_out_dir/"single_cells_0.json")

constructing single cells from json dict: 100%|██████████| 42/42 [00:00<00:00, 5673.27it/s]


In [20]:
loaded_scs[0]

SingleCellStatic(id=fc9d1413-2399-4b6f-bf91-13ba4e05b34c, timeframe=0, bbox=[ 143.  978.  207. 1044.])

In [21]:
for sc in single_cells[:2]:
    print(sc.id)

fc9d1413-2399-4b6f-bf91-13ba4e05b34c
402fbdcf-c641-45ca-a580-54fe5b7519e8


In [22]:
for sc in loaded_scs[:2]:
    print(sc.id)

fc9d1413-2399-4b6f-bf91-13ba4e05b34c
402fbdcf-c641-45ca-a580-54fe5b7519e8


In [23]:
for sc in single_cells:
    for loaded_sc in loaded_scs:
        if sc.id == loaded_sc.id:
            # compare contour, contours are np.array
            assert np.allclose(sc.contour, loaded_sc.contour), f"the difference between sc.contour and loaded_sc.contour is {sc.contour - loaded_sc.contour}, ids are {sc.id} and {loaded_sc.id}"

## SingleCellTrajectory: saving & loading

In [24]:
from typing import List
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)


sct_collection = track_SORT_bbox_from_scs(single_cells, dic_dataset, mask_dataset=mask_dataset, max_age=1, min_hits=1)

SORT tracking...: 100%|██████████| 3/3 [00:00<00:00, 42.01it/s]


In [25]:
sct_collection.write_json(io_out_dir/"sct_collection.json")

In [26]:
sct_collection.load_from_json_file(io_out_dir/"sct_collection.json")

|-----> json loaded from test_io_output/sct_collection.json
|-----> Creating SingleCellTrajectoryCollection from json_dict...


In [27]:
sct_collection.get_all_trajectories()[0].img_dataset

In [28]:
sct_collection.write_json(io_out_dir/"sct_collection.json", dataset_json_dir=io_out_dir/"another_dataset_folder")
sct_collection.load_from_json_file(io_out_dir/"sct_collection.json")

|-----> json loaded from test_io_output/sct_collection.json
|-----> Creating SingleCellTrajectoryCollection from json_dict...
